# Check Old Patience Prevalence

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import os
import random
import dill
import pickle
from tabulate import tabulate
import matplotlib.pyplot as plt


import sys

import warnings
warnings.filterwarnings("ignore")

In [2]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

if IN_COLAB:  
  # Mount the Google Drive at mount
  mount='/content/gdrive'
  print("Colab: mounting Google drive on ", mount)
  # connect your colab with the drive
  drive.mount(mount)

 # Switch to the directory on the Google Drive that you want to use
  import os
  path_to_repo = mount + "/My Drive/MIMIC-III Text Mining/mimim_iii_readmission"

else:
   path_to_repo = os.path.dirname(os.getcwd())

  
print(path_to_repo)

C:\Users\luca9\Documents\MIMIC-III Text Mining\mimim_iii_readmission


In [23]:
# PARAMETERS

session_seed = 42 # set seed for our session
include_val = False # set to True if we want to also create a validation set
tune_models = True # set to True if we want to perform parameter tuning

icu_stays = True # set to TRUE if we want to have only ICU stays
lemmatize = True # set to false if we want to do stemming
lemma_tag = str(np.where(lemmatize, "_lemma",""))
heavier_proc = True # if we want a heavier processing
if heavier_proc:
    heavier_tag = '_heavier'
else:
    heavier_tag = ''
    
spacy = True
if spacy: lemma_tag = str(np.where(lemmatize, "_lemma_spacy",""))

seed_tag = f'_{session_seed}'

halving = True # if we want to perform halving tune
if tune_models:
    if halving:
        tune_tag = '_tuned_halv'
    else:
        tune_tag = '_tuned'   
else:
    tune_tag = ''

random.seed(session_seed)

med_7 = False # set to True if we want to use our Med7 preprocessing

if med_7:
    med_tag = "_med7"
else:
    med_tag = ''
    
feat_select = False # select True if we want to use Lasso as a feature selection method

if feat_select:
    feat_tag = "_featselect"
else:
    feat_tag = ''
    
expanded_def = True # set to True if we want to consider future readmissions and avoid using CMS 

if icu_stays == True:
    icu_folder = 'icu_only'
    if expanded_def:
        icu_folder = 'expanded'
else:
    icu_folder = 'all_hosp'

In [4]:
path_to_data = os.path.join(path_to_repo, "data", icu_folder,"")
print(path_to_data)

C:\Users\luca9\Documents\MIMIC-III Text Mining\mimim_iii_readmission\data\expanded\


In [5]:
path_to_processed = os.path.join(path_to_data,"processed","")
os.makedirs(path_to_processed, exist_ok=True) # we create the directory if it does not exist
print(path_to_processed)

C:\Users\luca9\Documents\MIMIC-III Text Mining\mimim_iii_readmission\data\expanded\processed\


In [25]:
path_to_models = os.path.join(path_to_data,"models","")
os.makedirs(path_to_models, exist_ok=True) # we create the directory if it does not exist
print(path_to_models)

C:\Users\luca9\Documents\MIMIC-III Text Mining\mimim_iii_readmission\data\expanded\models\


In [6]:
df = pd.read_feather(os.path.join(path_to_data,f"df_cleaned{lemma_tag}{med_tag}{heavier_tag}"))

In [7]:
df.columns

Index(['index', 'subject_id', 'hadm_id', 'admittime', 'dischtime',
       'first_careunit', 'last_careunit', 'age', 'gender', 'marital_status',
       'insurance', 'diagnosis', 'text', 'next_readmit_dt', 'target', 'clean'],
      dtype='object')

In [8]:
df.age

0        76.526788
1        47.845044
2        65.940670
3        50.148292
4        39.866116
           ...    
42820    74.610874
42821    54.459307
42822    54.611874
42823    65.262831
42824    65.377831
Name: age, Length: 42825, dtype: float64

In [9]:
#%% Creating categorical age variable "agecat"

df.loc[(df["age"]<18), "agecat"] = "1(0-17 ans)"
df.loc[(df["age"]>=18) & (df["age"]<45), "agecat"] = "2(18-44 ans)"
df.loc[(df["age"]>=45) & (df["age"]<65), "agecat"] = "3(45-64 ans)"
df.loc[(df["age"]>=65) & (df["age"]<85), "agecat"] = "4(65-84 ans)"
df.loc[(df["age"]>=85), "agecat"] = "5(85 ans et plus)"

In [10]:
df.agecat.value_counts()

4(65-84 ans)         17283
3(45-64 ans)         15074
2(18-44 ans)          6387
5(85 ans et plus)     4081
Name: agecat, dtype: int64

In [11]:
df.marital_status.value_counts()

MARRIED              20594
SINGLE               11141
WIDOWED               5934
DIVORCED              2753
SEPARATED              487
UNKNOWN (DEFAULT)      268
LIFE PARTNER            15
Name: marital_status, dtype: int64

In [12]:
df.insurance.value_counts()

Medicare      23322
Private       13951
Medicaid       3904
Government     1231
Self Pay        417
Name: insurance, dtype: int64

In [13]:
df.gender.value_counts()

M    24170
F    18655
Name: gender, dtype: int64

In [14]:
df.columns

Index(['index', 'subject_id', 'hadm_id', 'admittime', 'dischtime',
       'first_careunit', 'last_careunit', 'age', 'gender', 'marital_status',
       'insurance', 'diagnosis', 'text', 'next_readmit_dt', 'target', 'clean',
       'agecat'],
      dtype='object')

In [15]:
# Features we want to test
categorical_test = ['agecat', 'gender', 'marital_status', 'insurance']
data = df.loc[:,categorical_test + ['target']]

In [16]:
#%% Test proportion difference

def two_prop_test(n1, n2, N1, N2):
    '''
    Implement a proportions difference test between two samples
    Large sample assumed (Z statistics), 2 tailed
    
    Parameters
    ----------
    n1 : counts of success for the first sample
    N1 : first sample size
    n2 : counts of success for the secons sample
    N2 : second sample
    
    Returns
    -------
    Statistic of test and p value
    '''
    
    import numpy as np
    from scipy.stats import norm
    
    p1 = n1/N1
    p2 = n2/N2
    p = (N1*p1 + N2*p2)/(N1+N2)
  
    Z_stat = (p1-p2)/np.sqrt(p*(1-p)/N1 + p*(1-p)/N2)
    p_value = 2*norm.cdf(-np.abs(Z_stat))
    
    return Z_stat, p_value

#%% Multivariate Mahalanobis distance

def mahalanobis(data):
    '''
    Computes the multivariate mahalanobis distance between treatment an control

    Parameters
    ----------
    data : contigency table of proportions
      
    Returns
    -------
    The mahalanobis distance

    '''

    from numpy.linalg import inv
    
    data = data.reset_index()
    T = data["(%+)"]
    C = data["(%-)"]
    
    S_size = data.shape[0]
    
    T = T[[i for i in range(1,S_size)]]
    C = C[[i for i in range(1,S_size)]]
       
    S = np.empty([S_size-1, S_size-1])
    
    for k in range(1,S_size) :
        for l in range(1,S_size):
            if k==l :
                S[k-1,l-1] = (T[k]*(1-T[k])+C[k]*(1-C[k]))/2
            else :
                S[k-1,l-1] = (C[k]*C[l]+T[k]*T[l])/2
    
    print(S)
    
    d_square = np.matmul(np.transpose(T-C),np.matmul(inv(S),(T-C)))
    return np.sqrt(d_square)

#%% Standardized difference

def std_diff(p1, p2):
    '''
    Computes Cohen's d on proportions

    Parameters
    ----------
    p1 : proportion of success in sample 1
    p2 : proportion of success in sample 2

    Returns
    -------
    None.

    '''
    import numpy as np
    d = (p1-p2)/np.sqrt((p1*(1-p1)+p2*(1-p2))/2)
    
    return d


#%% Implement chi-square tests

def los_table(data_quali):
    '''
    Generates a bivariate table with the LOS- or LOS+ as columns
    Then compare proportions with p values and standardized difference

    Parameters
    ----------
    data_quali : a catagorical pandas table with "dureecat" as outcome
    

    Returns
    -------
    Table

    '''

    from scipy.stats import chi2_contingency
    
    restable = []
    
    
    for col in categorical_test:
            
        contable = pd.crosstab(data_quali.loc[:,col], data_quali["target"], margins=True)        
        obs = np.array(contable.iloc[0:(contable.shape[0]-1),0:(contable.shape[1]-1)])       
        chisquare, p_value, df, exp = chi2_contingency(obs)        
        residuals = (obs - exp)/ np.sqrt(exp)       
        p = obs/np.sum(obs, axis=0)        
        phi = np.sqrt(chisquare/data_quali.shape[0])
        
        for i in range(obs.shape[0]) :
            
            Z_stat, mod_p_value = two_prop_test(obs[i,0], obs[i,1], np.sum(obs[:,0], 
                                                                           axis=0), np.sum(obs[:,1], axis=0))
            
            d = std_diff(p[i,1], p[i,0])
            
            restable.append([col, contable.index[i], np.sum(obs, axis=1)[i], 
                             obs[i,0], p[i,0], obs[i,1], p[i,1], p_value, phi, mod_p_value, d])
           
    resdata = pd.DataFrame(restable)
    resdata.columns = ["Variable", "Modality", "N", "NoReadm-", "(%-)", "Readm+", "(%+)", 
                       "p-value", "Size Effect (Phi)", "modality p-value", "Cohen's d"]
    
    return(resdata)   

In [17]:
bivariee = los_table(data)

In [18]:
bivariee

,Variable,Modality,N,NoReadm-,(%-),Readm+,(%+),p-value,Size Effect (Phi),modality p-value,Cohen's d
0,agecat,2(18-44 ans),6387,6035,0.149451,352,0.144026,3.694430e-01,0.008573,4.647097e-01,-0.015333
1,agecat,3(45-64 ans),15074,14237,0.352567,837,0.342471,3.694430e-01,0.008573,3.102244e-01,-0.021202
2,agecat,4(65-84 ans),17283,16255,0.402541,1028,0.420622,3.694430e-01,0.008573,7.686084e-02,0.036748
3,agecat,5(85 ans et plus),4081,3854,0.095441,227,0.092881,3.694430e-01,0.008573,6.755006e-01,-0.008767
4,gender,F,18655,17597,0.435774,1058,0.432897,7.967375e-01,0.001245,7.805691e-01,-0.005805
5,gender,M,24170,22784,0.564226,1386,0.567103,7.967375e-01,0.001245,7.805691e-01,0.005805
6,marital_status,DIVORCED,2753,2579,0.066512,174,0.071990,2.147904e-04,0.024679,2.954130e-01,0.021579
7,marital_status,LIFE PARTNER,15,15,0.000387,0,0.000000,2.147904e-04,0.024679,3.334765e-01,-0.027821
8,marital_status,MARRIED,20594,19468,0.502076,1126,0.465867,2.147904e-04,0.024679,5.517575e-04,-0.072503
9,marital_status,SEPARATED,487,446,0.011502,41,0.016963,2.147904e-04,0.024679,1.595709e-02,0.046116


In [19]:
bivariee.to_excel(f'{path_to_data}check_proportions.xlsx') 

## LDA Analysis

In [32]:
with open(f'{path_to_data}lda{seed_tag}{lemma_tag}{med_tag}{heavier_tag}', 'rb') as file: # and save the fitted model
    lda = dill.load(file)

In [28]:
def load_datasets(method, include_val = True, target = False):
    """
    Function to load train, test and validation set based on the chosen method
    method: string for the processing method we want to load
    include_diag: if we want to load the dataframes with the diagnosis text, default True
    include_test: if we want to load also the test set, default True
    target: if we are importing our target variables
    """
    global path_to_processed
    if target == True: 
        target = 'y_'
    else: 
        target = ''
    # load it back
    train = pd.read_feather(f'{path_to_processed}{target}train_{method}{seed_tag}{lemma_tag}{med_tag}{heavier_tag}')
    test = pd.read_feather(f'{path_to_processed}{target}test_{method}{seed_tag}{lemma_tag}{med_tag}{heavier_tag}')
    if include_val == True:
        val = pd.read_feather(f'{path_to_processed}{target}val_{method}{seed_tag}{lemma_tag}{med_tag}{heavier_tag}')
    else: val = []
    return train, test, val

def df_perm_importance(X_valid, result, inbuilt = False):
    """
    Function to create a dataframe from a permutation importance object
    """
    flabels = X_valid.columns
    if inbuilt == True:
      impvalues = 100*result/np.max(result) # if we are using the inbuilt function we do not need to call for the mean
    else:
      impvalues = 100*result.importances_mean/np.max(result.importances_mean)
    return pd.DataFrame({"features": flabels, "importance":impvalues}).sort_values("importance", ascending=False)

In [41]:
train, test_lda, val = load_datasets('lda', include_val = include_val)
train, test_freq, val = load_datasets('frequency', include_val = include_val)

In [26]:
with open(f'{path_to_models}_log_reg_lda{tune_tag}{seed_tag}{lemma_tag}{med_tag}{heavier_tag}{feat_tag}', 'rb') as file:
            estimator = dill.load(file)

In [44]:
lda_components = pd.DataFrame(lda.components_, columns = test_freq.columns, index = test_lda.columns)

In [45]:
df_imp = df_perm_importance(test_lda, estimator.coef_[0], inbuilt = True)

In [46]:
df_imp

,features,importance
168,F168,100.000000
141,F141,94.449794
248,F248,89.385835
56,F56,79.096350
261,F261,63.145862
...,...,...
163,F163,-44.192683
206,F206,-44.415818
72,F72,-58.631264
260,F260,-67.225070


In [47]:
lda_components

,aa,aaa,aaa repair,aado,aado req,aaox,ab,ab negative,abd,abd aorta,...,zinc sulfate,zocor,zofran,zoloft,zolpidem,zolpidem tablet,zone,zoster,zosyn,zyprexa
F0,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,14.106206,0.003333,...,0.003333,0.003333,0.003333,0.003333,7.244575,6.633666,0.003333,0.003333,1.247545,0.003333
F1,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,...,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333
F2,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.925080,0.003333,...,0.003333,1.356395,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333
F3,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,...,0.003333,0.003333,3.530548,0.003333,0.003333,0.003333,0.003333,0.003333,0.986577,3.897516
F4,7.010160,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,120.331585,0.003333,...,0.003333,10.751436,0.003333,12.505681,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F295,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,2.388277,0.003333,...,0.003333,0.003333,0.003333,0.003333,1.361024,0.264795,0.003333,0.003333,0.003333,0.003333
F296,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,10.112100,0.003333,...,0.003333,0.003333,0.003333,0.003333,22.335288,0.003333,0.003333,0.003333,0.679449,0.003333
F297,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,...,87.188722,0.003333,0.003333,10.262500,0.003333,0.003333,3.593326,0.003333,566.510345,0.003333
F298,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,0.003333,...,0.003333,65.627318,0.003333,12.407669,0.003333,0.003333,4.105000,1.793439,0.003333,0.003333


In [59]:
lda_components.loc[df_imp.features.iloc[0]].sort_values(ascending = False).head(20)

hd              10081.976979
dialysis         6194.959518
esrd             3847.342124
hemodialysis     3247.732217
tablet           2601.310519
renal            2215.952019
tid              1812.787017
catheter         1770.052960
daily            1733.046743
sig              1679.556589
esrd hd          1672.304287
capsule          1474.117873
meal             1245.301161
acid             1241.022642
folic            1237.072128
folic acid       1235.605109
complex          1221.482485
po tid           1214.593946
continue         1156.518467
sevelamer        1153.612701
Name: F168, dtype: float64

In [60]:
lda_components.loc[df_imp.features.iloc[1]].sort_values(ascending = False).head(20)

tube                16922.642404
feed                 6090.228961
tube feed            5156.872790
peg                  2032.096156
placement            2005.304585
place                1908.205926
feeding              1702.966108
tube place           1683.112868
gastrostomy          1618.601473
percutaneous         1529.858115
continue             1430.093091
wean                 1394.301620
tube placement       1385.373807
tracheostomy         1237.414596
cc                   1172.447156
ventilator           1067.364966
gastrostomy tube     1044.003333
peg tube             1024.502228
status               1008.896661
tolerate             1008.322363
Name: F141, dtype: float64

In [61]:
lda_components.loc[df_imp.features.iloc[2]].sort_values(ascending = False).head(20)

bipap                      2473.873875
sleep                      1661.051462
respiratory                1396.867321
pco                        1335.204886
cpap                       1160.148261
abg                        1107.584724
ph                         1090.729098
hypercarbic                1025.710339
failure                     893.074007
osa                         889.300717
respiratory failure         874.178118
hypercarbic respiratory     820.521596
apnea                       765.051898
base                        742.681658
sleep apnea                 636.994981
hypercarbia                 589.115306
po pco                      582.366453
pco ph                      551.923148
sleep study                 497.830045
hypoventilation             495.911863
Name: F248, dtype: float64

In [62]:
lda_components.loc[df_imp.features.iloc[3]].sort_values(ascending = False).head(20)

trach                  2212.280615
tablet                 1926.918449
peg                    1667.245053
sig                    1504.953593
tracheostomy           1398.037126
tube                   1370.151868
respiratory            1289.627032
rehab                  1138.170520
daily                  1081.258010
place                  1048.855544
vent                   1037.724304
prn                     981.984840
tablet po               960.913288
wean                    938.494892
tid                     932.987081
failure                 906.667288
placement               766.112290
respiratory failure     743.062663
bid                     742.562573
ventilator              732.441421
Name: F56, dtype: float64

In [63]:
writer = pd.ExcelWriter(f'{path_to_repo}\output_lda.xlsx')
startrow = 0
for i in range(0,10):
    
    lda_words = lda_components.loc[df_imp.features.iloc[i]].sort_values(ascending = False).head(20)
    lda_words.to_excel(writer,sheet_name='LDA', startrow=startrow +1)
    worksheet = writer.sheets['LDA']
    worksheet.write(startrow, 0, f'Topic {i}')
    startrow += (lda_words.shape[0] + 2)
        
writer.close()      